In [1]:
import os

In [2]:
%pwd

'/home/tejas/MLProj/Thyroid-Disease-Prediction/research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'/home/tejas/MLProj/Thyroid-Disease-Prediction'

In [5]:
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/tejas05in/Thyroid-Disease-Prediction.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "tejas05in"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "9efcb5c7b79d0e949378459b922b1462a80fa413"

In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [7]:
from ThyroidProject.constants import *
from ThyroidProject.utils.common import read_yaml , create_directories

In [8]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH,
                 schema_filepath=SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.GradientBoostedTreesLearner
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params=params,
            metric_file_name=config.metric_file_name,
            target_column=schema.name,
            mlflow_uri="https://dagshub.com/tejas05in/Thyroid-Disease-Prediction.mlflow"
        )

        return model_evaluation_config

In [15]:
import os
import pandas as pd
from urllib.parse import urlparse
import mlflow
from mlflow.models import infer_signature
import numpy as np
import tensorflow as tf
import tensorflow_decision_forests as tfdf
from ThyroidProject.utils.common import save_json

In [30]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
        
    

    def log_into_mlflow(self):
        mlflow.tensorflow.autolog()
        
        test_data = pd.read_csv(self.config.test_data_path)
        test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(
            test_data, label=self.config.target_column)
        model = tf.keras.models.load_model(self.config.model_path)
        model.compile(metrics=["accuracy"])
        evaluation = model.evaluate(test_ds, return_dict=True)

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_uri_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():

            # Saving metrics as local
            # scores = {"accuray": evaluation["accuracy"], "loss": evaluation["loss"]}
            save_json(path=Path(self.config.metric_file_name), data=evaluation)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("loss", evaluation["loss"])
            mlflow.log_metric("accuray",evaluation["accuracy"])
            
            # Model registry does not work with file store
            if tracking_uri_type_store != 'file':

                # Register the model
                # There are other ways to use the model registry , which depends on the use case
                # Please refer to the documentation for more information
                # https://mlflow.org/docs/latest/model-registry.html #api-workflow
                mlflow.tensorflow.log_model(
                    model, 'model', registered_model_name="GradientBoostedTreesModel", conda_env="conda.yaml")

            else:
                mlflow.tensorflow.log_model(model, 'model')

In [31]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2024-01-27 19:41:31,653:INFO:common:yaml file: config/config.yaml loaded successfully]
[2024-01-27 19:41:31,654:INFO:common:yaml file: params.yaml loaded successfully]
[2024-01-27 19:41:31,655:INFO:common:yaml file: schema.yaml loaded successfully]
[2024-01-27 19:41:31,656:INFO:common:created directory at :artifacts]
[2024-01-27 19:41:31,656:INFO:common:created directory at :artifacts/model_evaluation]
[2024-01-27 19:41:31,660:WARNING:tf_logging:Some of the feature names have been changed automatically to be compatible with SavedModels because fix_feature_names=True.]


[INFO 24-01-27 19:41:31.9027 IST kernel.cc:1233] Loading model from path artifacts/model_trainer/model/assets/ with prefix 30989f3147be440e
[INFO 24-01-27 19:41:31.9083 IST decision_forest.cc:660] Model loaded with 111 root(s), 4117 node(s), and 14 input feature(s).
[INFO 24-01-27 19:41:31.9083 IST abstract_model.cc:1344] Engine "GradientBoostedTreesGeneric" built
[INFO 24-01-27 19:41:31.9083 IST kernel.cc:1061] Use fast generic engine


[2024-01-27 19:41:32,147:WARNING:polymorphic_function:5 out of the last 9 calls to <function Model.make_test_function.<locals>.test_function at 0x7f57c0643920> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.]
2/2 [==============================] - 0s 7ms/step - loss: 0.0000e+00 - accuracy: 0.9865
[2024-01-27 19:41:32,635:DEBUG:connectionpool:https://dagshub.com:443 "POST /tejas05in/Thyroid-Disease-Prediction.mlflow/api/2.0/mlflow/runs/create HTTP/1.1" 200 None]
[2024-01-27 19:

2024/01/27 19:41:33 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2024-01-27 19:41:34,367:INFO:builder_impl:Assets written to: /tmp/tmp80iqeeya/model/data/model/assets]
[2024-01-27 19:41:34,979:DEBUG:connectionpool:https://dagshub.com:443 "GET /tejas05in/Thyroid-Disease-Prediction.mlflow/api/2.0/mlflow/runs/get?run_uuid=2d297254accf4d9bb68c9333afb18624&run_id=2d297254accf4d9bb68c9333afb18624 HTTP/1.1" 200 None]
[2024-01-27 19:41:35,595:DEBUG:connectionpool:https://dagshub.com:443 "PUT /tejas05in/Thyroid-Disease-Prediction.mlflow/api/2.0/mlflow-artifacts/artifacts/dbd409d79d1546fdbfc410acf1df7960/2d297254accf4d9bb68c9333afb18624/artifacts/model/MLmodel HTTP/1.1" 200 None]
[2024-01-27 19:41:36,106:DEBUG:connectionpool:https://dagshub.com:443 "PUT /tejas05in/Thyroid-Disease-Prediction.mlflow/api/2.0/mlflow-artifacts/artifacts/dbd409d79d1546fdbfc410acf1df7960/2d297254accf4d9bb68c9333afb18624/artifacts/model/conda.yaml HTTP/1.1" 200 None]
[2024-01-27 19:41:36,822:DEBUG:connectionpool:https://dagshub.com:443 "PUT /tejas05in/Thyroid-Disease-Prediction.mlfl

Registered model 'GradientBoostedTreesModel' already exists. Creating a new version of this model...


[2024-01-27 19:41:51,160:DEBUG:connectionpool:https://dagshub.com:443 "GET /tejas05in/Thyroid-Disease-Prediction.mlflow/api/2.0/mlflow/runs/get?run_uuid=2d297254accf4d9bb68c9333afb18624&run_id=2d297254accf4d9bb68c9333afb18624 HTTP/1.1" 200 None]
[2024-01-27 19:41:51,569:DEBUG:connectionpool:https://dagshub.com:443 "POST /tejas05in/Thyroid-Disease-Prediction.mlflow/api/2.0/mlflow/model-versions/create HTTP/1.1" 200 None]


2024/01/27 19:41:51 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: GradientBoostedTreesModel, version 2
Created version '2' of model 'GradientBoostedTreesModel'.


[2024-01-27 19:41:51,977:DEBUG:connectionpool:https://dagshub.com:443 "POST /tejas05in/Thyroid-Disease-Prediction.mlflow/api/2.0/mlflow/runs/update HTTP/1.1" 200 None]
